<a href="https://colab.research.google.com/github/Rogerio-mack/Machine-Learning-I/blob/main/ML6_professor_resolve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="Universidade Presbiteriana Mackenzie">
</head>

<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->


# **Selecionando Hiperparâmetros de um Modelo**
---


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.lines import Line2D
import seaborn as sns

# Bank note authentication

In [10]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00267/data_banknote_authentication.txt',header=None)

# Wavelet Transformed image atributtes
df.columns = ['variance','skewness','curtosis','entropy','class']
display(df.head())



,variance,skewness,curtosis,entropy,class
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [11]:
df['class'].value_counts()

0    762
1    610
Name: class, dtype: int64

# By scratch... 

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score

X = df.drop(columns=['class'])
y = df['class']

scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X) 

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=123)

scores_means = {}

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=123)

for k, d in [(k,d) for k in range(2,9) for d in ['euclidean','manhattan']]:
  
  clf = KNeighborsClassifier(n_neighbors = k, metric= d )

  acc_scores = cross_val_score(clf, X_train, y_train, cv = 5)             

  scores_means[(k,d)] = acc_scores.mean()
  
scores_means = pd.DataFrame(scores_means.values(), index=scores_means.keys()).reset_index()
scores_means.columns = ['k','metric','score']

# display(scores_means)
print('\nBest result:\n')
display(scores_means.nlargest(1,'score'))



Best result:



,k,metric,score
12,8,euclidean,0.998958


In [13]:
k = scores_means.nlargest(1,'score').k.values[0]
metric = scores_means.nlargest(1,'score').metric.values[0]

clf = KNeighborsClassifier(n_neighbors = k, metric= metric )
print(clf)
    
clf.fit(X_train, y_train)                 
        
print("accuracy: %0.3f" % clf.score(X_test,y_test),'\n')

KNeighborsClassifier(metric='euclidean', n_neighbors=8)
accuracy: 1.000 



# Usando o Grid Search





In [14]:
clf = KNeighborsClassifier()
clf.get_params

<bound method BaseEstimator.get_params of KNeighborsClassifier()>

In [15]:
from sklearn.model_selection import train_test_split
from sklearn import neighbors
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

X = df.drop(columns=['class'])
y = df['class']

scaler.fit(X)
X = scaler.transform(X) 

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=123)

base_estimator = KNeighborsClassifier()
param_grid = {'n_neighbors': [3,4,5,6,7,8,9,10], 'metric': ['euclidean','manhattan']}

clf = GridSearchCV(base_estimator, param_grid, cv=5, scoring='accuracy')
clf.fit(X_train, y_train)

# print(clf.cv_results_)
print(clf.best_estimator_)

print()
print("Detailed classification report:")
print()
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
print()



KNeighborsClassifier(metric='euclidean', n_neighbors=8)

Detailed classification report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       229
           1       1.00      1.00      1.00       183

    accuracy                           1.00       412
   macro avg       1.00      1.00      1.00       412
weighted avg       1.00      1.00      1.00       412




In [16]:
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X = df.drop(columns=['class'])
y = df['class']

scaler.fit(X)
X = scaler.transform(X) 

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=123)

base_estimator = RandomForestClassifier(random_state=123)
param_grid = {'n_estimators':[3,4,5,6],'criterion':['gini','entropy'],'max_depth':[2,3,4,5]}

clf = GridSearchCV(base_estimator, param_grid, cv=5, scoring='accuracy')
clf.fit(X_train, y_train)

print(clf.cv_results_)
print(clf.best_estimator_)

print()
print("Detailed classification report:")
print()
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
print()





{'mean_fit_time': array([0.01115232, 0.01344247, 0.01554117, 0.01828518, 0.0109005 ,
       0.01817365, 0.01919365, 0.02049699, 0.01411347, 0.01621442,
       0.01744623, 0.02025547, 0.01180091, 0.01564474, 0.01813836,
       0.02370467, 0.01608367, 0.01478896, 0.01833844, 0.02179909,
       0.01245532, 0.016575  , 0.02190118, 0.02337003, 0.01337209,
       0.01734252, 0.02218742, 0.01879025, 0.00946808, 0.0105288 ,
       0.01291361, 0.0148654 ]), 'std_fit_time': array([6.06407472e-04, 9.76414641e-05, 2.38160499e-04, 4.34931079e-04,
       6.95060142e-04, 6.41788084e-03, 3.18819077e-03, 8.40636411e-04,
       3.52417412e-03, 1.92278492e-03, 2.15571239e-04, 6.36855908e-04,
       5.08948843e-04, 1.61277374e-03, 7.40842811e-04, 3.60539014e-03,
       5.08665601e-03, 4.27204438e-04, 2.30522731e-04, 9.92287677e-04,
       2.19632075e-04, 4.84152015e-04, 3.57744793e-03, 6.74795309e-04,
       9.93343393e-04, 5.82035967e-04, 2.26707168e-03, 3.62204065e-03,
       1.14619300e-03, 4.37030998e